In [34]:
import os
import json
import pandas as pd
import ast
import shutil
import tarfile
import re

In [17]:
def count_files(path):
  files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
  total_files = len(files)
  return total_files

In [18]:
test_path = r'C:\\Users\\Admin\\OneDrive\\ILMU\\DATA SCIENCE\\PORTOFOLIO\\Text Summarization - Indonesia AI\\liputan6_data\\canonical\\dev'
dev_path = r'C:\\Users\\Admin\\OneDrive\\ILMU\\DATA SCIENCE\\PORTOFOLIO\\Text Summarization - Indonesia AI\\liputan6_data\\canonical\\test'
train_path = r'C:\\Users\\Admin\\OneDrive\\ILMU\\DATA SCIENCE\\PORTOFOLIO\\Text Summarization - Indonesia AI\\liputan6_data\\canonical\\train'

In [20]:
count_test = count_files(test_path)
count_dev = count_files(dev_path)
count_train = count_files(train_path)

In [21]:
display(count_test)
display(count_dev)
display(count_train)

10972

10972

193883

In [22]:
def extract_data_from_folder(folder_path):
    data_dict_list = []

    # Mendapatkan daftar file dalam folder
    file_list = os.listdir(folder_path)

    for file_name in file_list:
        if file_name.endswith('.json'):
            file_path = os.path.join(folder_path, file_name)

            # Membaca file JSON
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)

                # Mengambil clean_article, clean_summary, dan extractive_summary
                clean_article = ' '.join(map(str, data.get('clean_article', []))).strip()
                clean_summary = ' '.join(map(str, data.get('clean_summary', []))).strip()
                extractive_summary = ' '.join(map(str, data.get('extractive_summary', []))).strip()

                # Menambahkan ke dictionary
                data_dict = {
                    'clean_article': clean_article,
                    'clean_summary': clean_summary,
                    'extractive_summary': extractive_summary
                }

                data_dict_list.append(data_dict)

    return data_dict_list


def extract_json_file(folder_path):
    converted_data = []

    # Loop through each file in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".json"):
            file_path = os.path.join(folder_path, filename)

            # Read JSON file
            with open(file_path, 'r') as file:
                data = json.load(file)

            # Convert data format
            converted_item = {
                'clean_article': ' '.join([' '.join(sentence) for sentence in data['clean_article']]),
                'clean_summary': ' '.join([' '.join(sentence) for sentence in data['clean_summary']])
            }

            # Append to the result list
            converted_data.append(converted_item)

    return converted_data

In [24]:
test = extract_json_file(test_path)

In [25]:
dev = extract_json_file(dev_path)

In [26]:

train = extract_json_file(train_path)

In [27]:
print(test[1])
print(dev[1])
print(train[1])

{'clean_article': 'Liputan6 . com , Jakarta : Diperkirakan 11 juta penduduk Indonesia positif terjangkit virus Hepatitis B . Bahkan , sepertiga di antaranya penderita kronis aktif yang bisa berakibat pengerasan hati dan kanker hati . Alhasil , satu dari 20 orang Indonesia diperkirakan menderita penyakit yang mematikan itu . Penegasan ini disampaikan Dr Udin Gunawan dari Perkumpulan Peneliti Hati Indonesia atau PPHI , baru-baru ini , di Jakarta . Dikatakan Udin , besarnya angka tersebut disebabkan rendahnya kesadaran masyarakat menjaga kesehatan dan minimnya pengetahuan masyarakat terhadap penyakit mematikan itu . Apalagi , gejala penyakit ini seringkali tak kelihatan dan baru dapat terdeteksi tiga sampai enam bulan setelah kontak dengan penderita terjadi . Karena itu , Udin meminta masyarakat melakukan deteksi dini terhadap penyebaran penyakit tersebut . Caranya , dengan melihat gejala klinis yang muncul , seperti merasa lemas , lelah , mual , kehilangan nafsu makan dan warna air kenci

In [35]:
def remove_spacing_before_punctuation(text):
    # Remove spaces before commas and periods
    text_without_spacing = re.sub(r'\s*([,.])', r'\1', text)
    return text_without_spacing

def clean_data(data):
    cleaned_data = []
    for item in data:
        cleaned_item = {key: remove_spacing_before_punctuation(value) for key, value in item.items()}
        # Change "Liputan6 . com" to "Liputan6.com"
        cleaned_item['clean_article'] = cleaned_item['clean_article'].replace('Liputan6 . com', 'Liputan6.com')
        cleaned_data.append(cleaned_item)
    return cleaned_data

In [41]:
cleaned_test = clean_data(test)
cleaned_dev = clean_data(dev)
cleaned_train = clean_data(train)

In [42]:
df_test = pd.DataFrame(cleaned_test)
df_dev = pd.DataFrame(cleaned_dev)
df_train = pd.DataFrame(cleaned_train)

In [43]:
display(df_test.head())
display(df_dev.head())
display(df_train.head())

,clean_article,clean_summary
0,"Liputan6. com, Jakarta : Pemerintah masih memb...",Pemerintah memberikan tenggat 14 hari kepada p...
1,"Liputan6. com, Jakarta : Diperkirakan 11 juta ...",Satu dari 20 orang Indonesia diperkirakan meng...
2,"Liputan6. com, Banda Aceh : Aksi peledakan kan...",Peledakan bom kembali terjadi di Aceh. Kali in...
3,"Liputan6. com, Surabaya : Petugas Kepolisian R...",Polres Surabaya Timur menangkap seorang penged...
4,"Liputan6. com, Jakarta : Yogyakarta dan Bali m...",Lima seniman Yogyakarta dan Bali menggelar pam...


,clean_article,clean_summary
0,"Liputan6. com, Jakarta : Kepolisian Daerah Ria...",Kapolda Riau baru Brigjen Pol. Johny Yodjana b...
1,"Liputan6. com, Jakarta : Bank Indonesia dinila...",Kendati Bank Sentral AS menurunkan suku bungan...
2,"Liputan6. com, Jakarta : Berbagai kendala meng...",Pemerintah bermaksud akan lebih mengandalkan s...
3,"Liputan6. com, Jakarta : Penghapusan beberapa ...","Revisi Kepmennaker Nomor 78 Tahun 2001, dinila..."
4,"Liputan6. com, Jakarta : Operasi Sadar Jaya ya...",Polisi menangkap 32 pengunjung Diskotik Mileni...


,clean_article,clean_summary
0,"Liputan6. com, Jakarta : Presiden Susilo Bamba...","Menurut Presiden Susilo Bambang Yudhoyono, kem..."
1,"Liputan6. com, Jakarta : Perdana Menteri Jepan...",Pada masa silam Jepang terlalu ambisius untuk ...
2,"Liputan6. com, Kutai : Banjir dengan ketinggia...",Puluhan hektare areal persawahan yang sebagian...
3,"Liputan6. com, Jakarta : Presiden Susilo Bamba...",Sekjen PBB Kofi Annan memuji langkah Presiden ...
4,"Liputan6. com, Solok : Warga Kampung Batu Dala...",Untuk mempercepat pelaksanaan belajar-mengajar...


In [44]:
df_test.to_csv('./df_test.csv')
df_dev.to_csv('./df_dev.csv')
df_train.to_csv('./df_train.csv')